## Schema 3: Admissions

![Cube 3](img/Cube3.png)

In [2]:
# Libraries and connections

require 'sequel'

DB_MIMIC = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['mimiciii']
                          )

DB_OLAP = Sequel.connect(adapter: 'postgres',
                          host: 'postgres',
                          user: 'postgres',
                          password: 'password',
                          database: 'mimic3_demo',
                          search_path: ['olap']
                          )

load '../etl/model.rb'
nil

#### Time dimension table

In [18]:
# Creation
DB_OLAP.drop_table?(:admission_intime)
DB_OLAP.create_table(:admission_intime) do
  String :day, primary_key: true
  String :month
  String :year
end

# Population
Admission.all.uniq.each do |admission|
  time = admission.admittime
  year = time.year
  month = time.month
  day = time.day
  day_str = "#{year}-#{month}-#{day}"
  if DB_OLAP[:admission_intime].where(day: day_str).empty?
    DB_OLAP[:admission_intime].insert(
      day: day_str,
      month: "#{year}-#{month}",
      year: year.to_s
    )
  end
end

nil

#### Death dimension table

In [8]:
# Creation
DB_OLAP.drop_table?(:admission_death)
DB_OLAP.create_table(:admission_death) do
  Integer :hospital_expire_flag, primary_key: true
  String :status
end

# Population
DB_OLAP[:admission_death].insert(
      hospital_expire_flag: 0,
      status: "alive"
)

DB_OLAP[:admission_death].insert(
      hospital_expire_flag: 1,
      status: "dead"
)


nil

#### Patient age dimension table

In [12]:
def age_in_completed_years (bd, d)
    # Difference in years, less one if you have not had a birthday this year.
    a = d.year - bd.year
    a = a - 1 if (
         bd.month >  d.month or 
        (bd.month >= d.month and bd.day > d.day)
    )
    a
end

:age_in_completed_years

In [13]:
# Creation
DB_OLAP.drop_table?(:admission_pat_age)
DB_OLAP.create_table(:admission_pat_age) do
  Integer :age, primary_key: true
  String :decade
end

# Population
Admission.all.uniq.each do |admission|
  age = age_in_completed_years(admission.patient.dob, admission.admittime)
  decade = age / 10
  decade_str = "#{decade}0 - #{decade}9"  
  if DB_OLAP[:admission_pat_age].where(age: age).empty?
    DB_OLAP[:admission_pat_age].insert(
      age: age,
      decade: decade_str
    )
  end
end

nil

#### Patient status dimension table

In [14]:
# Creation
DB_OLAP.drop_table?(:admission_pat_status)
DB_OLAP.create_table(:admission_pat_status) do
  String :status, primary_key: true
  String :gender
  String :ethnicity
  String :insurance
end

# Population
Admission.all.uniq.each do |admission|
  ethnicity = admission.ethnicity
  insurance = admission.insurance
  gender = admission.patient.gender
  status = "#{gender} - #{ethnicity} - #{insurance}"
  
  if DB_OLAP[:admission_pat_status].where(status: status).empty?
    DB_OLAP[:admission_pat_status].insert(
      status: status,
      gender: gender,
      ethnicity: ethnicity,
      insurance: insurance
    )
  end
end

nil

#### Fact table

In [21]:
# Creation
DB_OLAP.drop_table?(:admissions)
DB_OLAP.create_table(:admissions) do
  Integer :id, primary_key: true
  Integer :diagnosis_count
  Integer :output_events_count
  
  Integer :hospital_expire_flag
  Integer :age
  String :status
  String :intime
end

# Population
Admission.all.uniq.each do |admission|
  diagnosis_count = admission.diagnosis.length
  output_events_count = admission.outputevents.length
  DB_OLAP[:admissions].insert(
    id: admission.row_id,
    diagnosis_count: diagnosis_count,
    output_events_count: output_events_count,
    
    hospital_expire_flag: admission.hospital_expire_flag,
    age: age_in_completed_years(admission.patient.dob, admission.admittime),
    status: "#{admission.patient.gender} - #{admission.ethnicity} - #{admission.insurance}",
    intime: "#{admission.admittime.year}-#{admission.admittime.month}-#{admission.admittime.day}"
  )
end

nil

## XML implementation